In [160]:
# %matplotlib inline
# import mpld3
# mpld3.enable_notebook()
import rosbag_pandas
import pandas as pd
import matplotlib.pyplot as plt
import easygui
import rosbag
import json_tricks as json
import numpy as np
import cPickle as pkl
import time
import seaborn as sns
def TicTocGenerator():
    # Generator that returns time differences
    ti = 0           # initial time
    tf = time.time() # final time
    while True:
        ti = tf
        tf = time.time()
        yield tf-ti # returns the time difference

TicToc = TicTocGenerator() # create an instance of the TicTocGen generator

# This will be the main function through which we define both tic() and toc()
def toc(tempBool=True):
    # Prints the time difference yielded by generator instance TicToc
    tempTimeInterval = next(TicToc)
    if tempBool:
        print( "Elapsed time: %f seconds.\n" %tempTimeInterval )

def tic():
    # Records a time in TicToc, marks the beginning of a time interval
    toc(False)
    
def pickler( obj, path):
    """
    Pickle a Python object
    """
    with open(path, "wb") as pfile:
        pkl.dump(obj, pfile)

def depickler( path):
    """
    Extracts a pickled Python object and returns it
    """
    with open(path, "rb") as pfile:
        data = pkl.load(pfile)
    return data


In [173]:
dff['metadata']['parameters']

TypeError: 'NoneType' object has no attribute '__getitem__'

In [187]:
'''Load a dataframe from pickle and analyse lr cases'''

defaultPath="/home/rhagoletis/catkin/src/World/bags/"
path=easygui.fileopenbox(title="Choose pickle Dataframes"
                          ,default=defaultPath,
                          multiple=False,filetypes=["*.pickle"])
print path

dff=pd.read_pickle(path)
df=dff['df']

/home/rhagoletis/catkin/src/World/bags/2017_01_02/2017-01-02__18:07:00_apple11_01_traj.bag_df.pickle


In [188]:
# row and column sharing
f, ((ax2, ax1), (ax3, ax4)) = plt.subplots(2, 2, sharex=True, sharey=True,figsize=(16,16))
ax={4:ax4,1:ax1,2:ax2,3:ax3}

# ax1.plot(x, y)
# ax1.set_title('lr anal')
for i in range(int(df.trajectory__case.max()+1)):
    df2=df[(df.trajectory__case==i) & (df.trajectory__runNum<20) &(df.trajectory__isFlying==True)]
    ax[i+1].scatter(df2.trajectory__pPos_x,df2.trajectory__pPos_y,lw=0,alpha=0.1)
    ax[i+1].set_aspect('equal', 'datalim')

    print i
plt.tight_layout()
plt.show()

# ax2.scatter(df2.trajectory__pPos_x,df2.trajectory__pPos_y,lw=0)
# plt.show()
# ax3.scatter(x, 2 * y ** 2 - 1, color='r')
# ax4.plot(x, 2 * y ** 2 - 1, color='r')


0
1
2
3


In [81]:
print df2.trajectory__o1Pos_x[0],df2.trajectory__o1Pos_y[0]
print df2.trajectory__o2Pos_x[0],df2.trajectory__o2Pos_y[0]

0.0 0.0
0.0 0.0


In [72]:
df2.trajectory__o1Pos_x.plot()
df2.trajectory__o2Pos_x.plot()
df2.trajectory__o1Pos_y.plot()
df2.trajectory__o2Pos_y.plot()
plt.show()

In [96]:
df2=df[df.trajectory__case==3]
# df2

In [97]:
plt.scatter(df2.trajectory__pPos_x,df2.trajectory__pPos_y,lw=0)
plt.show()

In [106]:
range(3)

[0, 1, 2]

In [85]:
df.trajectory__case.max()

3.0

In [66]:
jsonPath=paths[0].split('.bag')[0]+'.json'
jsonPath

'/home/rhagoletis/catkin/src/World/bags/2017_01_02/2017-01-02__15:45:54_apple11_01_traj.json'

In [44]:
with open(jsonPath,'r') as js:
    a= js.read()

In [46]:
json.loads(a)

ValueError: No JSON object could be decoded